This script retrieves the list of luftdaten sensors that are currently delivering data.
It then filter the sensors list on a fiven boundary (bbox) and store the list into an sqlite database.

In [1]:
import pandas as pd
import requests
import config
import sqlalchemy

In [2]:
engine = sqlalchemy.create_engine(f"sqlite:///{config.DB_PATH}")

In [3]:
# all currently active sensors on lufdaten with location,sensor type:
URL_24h =  "https://maps.luftdaten.info/data/v2/data.24h.json"

In [4]:
r = requests.get(URL_24h).json()
df_geo = pd.DataFrame(r)
df_geo_location=df_geo.location.apply(pd.Series).add_prefix('location_')
df_geo = pd.concat([df_geo,df_geo_location],axis=1)

## Define the sensors geo boundaries (Berlin & Brandenburg)

In [5]:
bbox = [11.074219,51.282535,15.227051,53.651150]
df_geo.location_longitude = df_geo.location_longitude.astype(float)
df_geo.location_latitude = df_geo.location_latitude.astype(float)
df_geo_BB = df_geo[(df_geo.location_longitude>bbox[0]) &(df_geo.location_longitude<bbox[2])&(df_geo.location_latitude>bbox[1]) &(df_geo.location_latitude<bbox[3]) ]

In [6]:
df_sensor_type = df_geo_BB.sensor.apply(pd.Series).sensor_type.apply(pd.Series).add_prefix('sensor_type_')
df_sensor_id = df_geo_BB.sensor.apply(pd.Series)[['id']].add_prefix('sensor_')

In [7]:
df_geo_BB = pd.concat([df_geo_BB,df_sensor_id,df_sensor_type],axis=1)

In [8]:
df_geo_BB.drop(['location','sensor','sensordatavalues'],axis=1,inplace=True)

In [9]:
df_geo_BB.head()

,id,sampling_rate,timestamp,location_id,location_latitude,location_longitude,location_altitude,location_country,location_exact_location,location_indoor,sensor_id,sensor_type_id,sensor_type_name,sensor_type_manufacturer
68,4028026044,None,2019-06-25 05:38:19,125,52.516,13.426,37.0,DE,0,0,283,14,SDS011,Nova Fitness
94,4028028593,None,2019-06-25 05:38:53,150,52.352,14.058,41.4,DE,0,0,325,14,SDS011,Nova Fitness
95,4028028715,None,2019-06-25 05:38:54,150,52.352,14.058,41.4,DE,0,0,326,9,DHT22,various
96,4028040177,None,2019-06-25 05:40:17,5879,52.534,13.350,38.1,DE,0,0,327,14,SDS011,Nova Fitness
97,4028040250,None,2019-06-25 05:40:17,5879,52.534,13.350,38.1,DE,0,0,328,9,DHT22,various


In [10]:
df_geo_BB.to_sql(name='sensors_BB', con=engine,if_exists='replace')